# 6. Beam Search and Advanced Optimization

Everything we have done so far (Greedy, Temperature, Top-K, Top-P) has been **Local Optimization**.

At every single step, we look *only* at the very next token and try to make the best choice. This is fast, but it can lead us down a blind alley. What if choosing a slightly sub-optimal token *now* opens up a highly probable, brilliant grammatical structure three words from now?

**Beam Search** is a form of **Global Sequence Optimization**.

Instead of keeping just one sequence, we maintain $B$ active "beams" (hypotheses). At each step, we expand all $B$ beams with all possible next tokens, score the resulting sentences by multiplying all probabilities together, and keep only the top $B$ sentences to continue.

In [1]:
import torch
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer
import math

device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'

model_id = "Qwen/Qwen2.5-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)
model.eval()

Loading weights:   0%|          | 0/290 [00:00<?, ?it/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2

## Step 1: Why Log-Probabilities?

If you multiply probabilities together (e.g., `0.1 * 0.1 * 0.1`), the number quickly becomes infinitesimally small, leading to floating-point underflow on computers.

Instead of multiplying probabilities, we **add** their logarithms.

$\log(A \times B) = \log(A) + \log(B)$

Because probabilities are between 0 and 1, their logs are always negative. The closer to 0 (the less negative), the more probable the sequence.

In [2]:
probability_1 = 0.5
probability_2 = 0.1

# Multiplication
combined_prob = probability_1 * probability_2
print(f"Raw Combined Prob: {combined_prob}")

# Log Addition
log_prob_1 = math.log(probability_1)
log_prob_2 = math.log(probability_2)

combined_log_prob = log_prob_1 + log_prob_2
print(f"Log Combined Prob: {combined_log_prob} (Notice it is negative!)")

Raw Combined Prob: 0.05
Log Combined Prob: -2.995732273553991 (Notice it is negative!)


## Step 2: Implementing a simple Beam Search

Let's implement a simplified Beam Search. 

Note that true production Beam Search relies heavily on batching tensors to be fast. We will write a more readable, looped version to understand the exact logic.

In [3]:
def simplified_beam_search(prompt, beam_width=3, max_length=10, length_penalty=1.0):
    print(f"Prompt: '{prompt}' (Beam Width: {beam_width})")
    
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    
    # A "beam" is a tuple: (sequence_tensor, cumulative_log_prob)
    # We start with 1 beam: our initial prompt, with a starting score of 0.0
    beams = [(input_ids, 0.0)]
    
    for step in range(max_length):
        all_candidates = []
        
        # 1. Expand each active beam
        for seq, score in beams:
            with torch.no_grad():
                outputs = model(seq)
                
            # Get the logits for the last token
            logits = outputs.logits[0, -1, :]
            
            # Convert to log probabilities (`log_softmax` is numerically stable)
            log_probs = F.log_softmax(logits, dim=-1)
            
            # To save massive compute, we don't actually expand the entire vocabulary.
            # We only expand the top B most likely next tokens from this specific beam.
            top_k_log_probs, top_k_indices = torch.topk(log_probs, beam_width)
            
            # 2. Gather candidates from this expansion
            for i in range(beam_width):
                next_token_id = top_k_indices[i].unsqueeze(0).unsqueeze(0)
                token_log_prob = top_k_log_probs[i].item()
                
                # Append token to sequence
                new_seq = torch.cat([seq, next_token_id], dim=-1)
                
                # Add the log prob (which is negative) to our running score
                new_score = score + token_log_prob
                
                all_candidates.append((new_seq, new_score))
                
        # 3. Sort all candidates by score (descending)
        # Apply length penalty to prevent the model from artificially favoring shorter sentences
        # Score / (Length ^ penalty)
        all_candidates = sorted(all_candidates, key=lambda x: x[1] / (x[0].shape[1] ** length_penalty), reverse=True)
        
        # 4. Prune back down to `beam_width`
        beams = all_candidates[:beam_width]
        
        # Print debug info to watch the beams fight for survival
        print(f"\n--- Step {step+1} top candidates ---")
        for i, (seq, score) in enumerate(beams):
            text = tokenizer.decode(seq[0])
            print(f"Beam {i+1} [Score: {score:.4f}]: {repr(text)}")
            
    print("\n\n=== Final Winner ===")
    final_seq, final_score = beams[0]
    return tokenizer.decode(final_seq[0])

# Try it with a beam width of 1 (which equals Greedy decoding)
simplified_beam_search("The chef prepared a", beam_width=1, max_length=5)

print("\n===========================\n")

# Try it with a beam width of 3
simplified_beam_search("The chef prepared a", beam_width=3, max_length=5)

Prompt: 'The chef prepared a' (Beam Width: 1)

--- Step 1 top candidates ---
Beam 1 [Score: -0.4707]: 'The chef prepared a large'

--- Step 2 top candidates ---
Beam 1 [Score: -0.8828]: 'The chef prepared a large batch'

--- Step 3 top candidates ---
Beam 1 [Score: -0.8892]: 'The chef prepared a large batch of'

--- Step 4 top candidates ---
Beam 1 [Score: -3.2017]: 'The chef prepared a large batch of cookies'

--- Step 5 top candidates ---
Beam 1 [Score: -4.0376]: 'The chef prepared a large batch of cookies for'


=== Final Winner ===


Prompt: 'The chef prepared a' (Beam Width: 3)

--- Step 1 top candidates ---
Beam 1 [Score: -0.4707]: 'The chef prepared a large'
Beam 2 [Score: -2.2188]: 'The chef prepared a batch'
Beam 3 [Score: -3.3438]: 'The chef prepared a total'

--- Step 2 top candidates ---
Beam 1 [Score: -0.8828]: 'The chef prepared a large batch'
Beam 2 [Score: -2.2251]: 'The chef prepared a batch of'
Beam 3 [Score: -2.6270]: 'The chef prepared a large number'

--- Step 3 to

'The chef prepared a large batch of lasagna'

## 🔬 Experimentation Ideas

1. **Compare Beam Width = 1 (greedy), 3, 5, 10:**
   * *Notice how execution time slows down dramatically as the tree branches out.*
2. **Track cumulative log-probability of each beam:**
   * *Look at the debug prints at Step 5. Did the sequence that was winning at Step 1 actually end up being the final winner? (This proves why Beam Search works where Greedy fails).* 
3. **Compare Beam Search vs Top-P for:**
   * *Translation-style prompts: "Translate English to French: Hello World ->"*
   * *Creative storytelling: "Write a poem about a robot:"*
   * *(Hint: Beam Search is heavily preferred for deterministic tasks like translation, but Top-P is preferred for creative chatting. Why?)*
4. **Experiment with length penalty values (0.6, 1.0, 1.5):**
   * *Modify the lambda func in step 3. High penalty forces longer sentences. Low penalty favors short ones. Observe when beam search produces shorter outputs.*

In [4]:
import torch
import torch.nn.functional as F
import time
import sys
import os
from transformers import AutoModelForCausalLM, AutoTokenizer

device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'
model_id = "Qwen/Qwen2.5-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

# Local, independent function for the cell
def simplified_beam_search_local(prompt, beam_width=3, max_length=10, length_penalty=1.0):
    print(f"\nPrompt: '{prompt}' (Beam Width: {beam_width}, Length Pen: {length_penalty})")
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    beams = [(input_ids, 0.0)]
    
    for step in range(max_length):
        all_candidates = []
        for seq, score in beams:
            with torch.no_grad():
                outputs = model(seq)
            log_probs = F.log_softmax(outputs.logits[0, -1, :], dim=-1)
            top_k_log_probs, top_k_indices = torch.topk(log_probs, beam_width)
            
            for i in range(beam_width):
                new_seq = torch.cat([seq, top_k_indices[i].unsqueeze(0).unsqueeze(0)], dim=-1)
                new_score = score + top_k_log_probs[i].item()
                all_candidates.append((new_seq, new_score))
                
        # The length penalty experiment is here:
        all_candidates = sorted(all_candidates, key=lambda x: x[1] / (x[0].shape[1] ** length_penalty), reverse=True)
        beams = all_candidates[:beam_width]
        
    final_seq, final_score = beams[0]
    print(f"Final Output: {tokenizer.decode(final_seq[0])}")
    return tokenizer.decode(final_seq[0])

print("=== Experiment 1: Execution Time vs Beam Width ===")
test_prompt = "The key to making a great pizza is"
for bw in [1, 3, 5]:
    start_time = time.time()
    simplified_beam_search_local(test_prompt, beam_width=bw, max_length=15)
    print(f"-> Time Taken: {time.time() - start_time:.2f} seconds")

print("\n=== Experiment 2 & 3: Translation vs Creative ===")
print("Top-P excels at Creative tasks due to randomness. Beam excels at Translation because there is usually one 'optimum' path.")
translation_prompt = "Translate English to French: Hello World ->"
creative_prompt = "Write a short poem about a robot:"

print("\n--- Translation with Beam (Usually accurate) ---")
simplified_beam_search_local(translation_prompt, beam_width=5, max_length=10)

print("\n--- Creative with Beam (Usually boring/repetitive) ---")
simplified_beam_search_local(creative_prompt, beam_width=5, max_length=15)

print("\n=== Experiment 4: Length Penalties ===")
print("High penalty (>1.0) artificially boosts scores of long sentences. Low penalty (<1.0) punishes them.")
print("\nLength Penalty 0.6 (Favors Short):")
simplified_beam_search_local(test_prompt, beam_width=3, max_length=20, length_penalty=0.6)

print("\nLength Penalty 1.5 (Favors Long):")
simplified_beam_search_local(test_prompt, beam_width=3, max_length=20, length_penalty=1.5)


Loading weights:   0%|          | 0/290 [00:00<?, ?it/s]

=== Experiment 1: Execution Time vs Beam Width ===

Prompt: 'The key to making a great pizza is' (Beam Width: 1, Length Pen: 1.0)
Final Output: The key to making a great pizza is the right dough. The dough is the base of the pizza, and it
-> Time Taken: 7.12 seconds

Prompt: 'The key to making a great pizza is' (Beam Width: 3, Length Pen: 1.0)
Final Output: The key to making a great pizza is to have the right ingredients, the right proportions, and the right technique.
-> Time Taken: 3.28 seconds

Prompt: 'The key to making a great pizza is' (Beam Width: 5, Length Pen: 1.0)
Final Output: The key to making a great pizza is using the right ingredients and techniques. Here are some tips to help you create
-> Time Taken: 6.27 seconds

=== Experiment 2 & 3: Translation vs Creative ===
Top-P excels at Creative tasks due to randomness. Beam excels at Translation because there is usually one 'optimum' path.

--- Translation with Beam (Usually accurate) ---

Prompt: 'Translate English to French

'The key to making a great pizza is to have the right ingredients, the right proportions, and the right way to cook it. Here are'